<a href="https://colab.research.google.com/github/Garg2111/Anomly_detection/blob/main/anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pycaret[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 50.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of flask to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44

In [1]:
import pycaret
pycaret.__version__

'3.3.2'

In [2]:
from pycaret.datasets import get_data
data = get_data('anomaly')

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10
0,0.263995,0.764929,0.138424,0.935242,0.605867,0.518790,0.912225,0.608234,0.723782,0.733591
1,0.546092,0.653975,0.065575,0.227772,0.845269,0.837066,0.272379,0.331679,0.429297,0.367422
2,0.336714,0.538842,0.192801,0.553563,0.074515,0.332993,0.365792,0.861309,0.899017,0.088600
3,0.092108,0.995017,0.014465,0.176371,0.241530,0.514724,0.562208,0.158963,0.073715,0.208463
4,0.325261,0.805968,0.957033,0.331665,0.307923,0.355315,0.501899,0.558449,0.885169,0.182754


In [3]:
from pycaret.anomaly import *
s = setup(data, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(1000, 10)"
2,Transformed data shape,"(1000, 10)"
3,Numeric features,10
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [4]:
from pycaret.anomaly import AnomalyExperiment
exp = AnomalyExperiment()

exp.setup(data, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(1000, 10)"
2,Transformed data shape,"(1000, 10)"
3,Numeric features,10
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [5]:
m = models()
print(m)

                                        Name  \
ID                                             
abod            Angle-base Outlier Detection   
cluster       Clustering-Based Local Outlier   
cof         Connectivity-Based Local Outlier   
iforest                     Isolation Forest   
histogram  Histogram-based Outlier Detection   
knn             K-Nearest Neighbors Detector   
lof                     Local Outlier Factor   
svm                   One-class SVM detector   
pca             Principal Component Analysis   
mcd           Minimum Covariance Determinant   
sod               Subspace Outlier Detection   
sos             Stochastic Outlier Selection   

                                                  Reference  
ID                                                           
abod                                  pyod.models.abod.ABOD  
cluster    pycaret.internal.patches.pyod.CBLOFForceToDouble  
cof                                     pyod.models.cof.COF  
iforest          

In [6]:
all_models = list(m.index)
all_models

['abod',
 'cluster',
 'cof',
 'iforest',
 'histogram',
 'knn',
 'lof',
 'svm',
 'pca',
 'mcd',
 'sod',
 'sos']

In [10]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

results = []
for m in all_models:
    if(m=='sod'):
      continue
    model = exp.create_model(m)
    anomalies = exp.assign_model(model)
    pred = exp.predict_model(model, data=data)

    anomaly_count = pred[pred['Anomaly'] == 1].shape[0]

    filtered_data = pred[pred['Anomaly'] == 0]
    X = filtered_data.drop(columns=['Anomaly', 'Anomaly_Score'], errors='ignore')

    scaler = StandardScaler()
    pred_scaled = scaler.fit_transform(X)


    best_score = -1
    best_n_clusters = 0

    for nc in range(2, 21):
        kmeans = KMeans(n_clusters=nc, random_state=123)
        cluster_labels = kmeans.fit_predict(pred_scaled)
        score = silhouette_score(pred_scaled, cluster_labels)

        if score > best_score:
            best_score = score
            best_n_clusters = nc

    best_score = round(best_score, 4)
    results.append({'model': m, 'anamoly_count' : anomaly_count, 'best_n_clusters': best_n_clusters,
                    'best_silhouette_score': best_score})

results_df = pd.DataFrame(results)
print(results_df)

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

        model  anamoly_count  best_n_clusters  best_silhouette_score
0        abod              0                2                 0.2097
1     cluster             50                2                 0.2258
2         cof             50                2                 0.2248
3     iforest             50                3                 0.2128
4   histogram             50                3                 0.2150
5         knn             46                2                 0.2246
6         lof             46                2                 0.2248
7         svm             50                3                 0.2088
8         pca             50                3                 0.2092
9         mcd             50                2                 0.2250
10        sos             50                2                 0.2195


In [11]:
results_df.to_csv('results.csv', index=False)